In [19]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import pylab

print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

numpy:  1.17.2
pandas:  1.0.3


In [20]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=18000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

b'Skipping line 17: expected 4 fields, saw 5\nSkipping line 33: expected 4 fields, saw 5\nSkipping line 44: expected 4 fields, saw 5\nSkipping line 58: expected 4 fields, saw 5\nSkipping line 61: expected 4 fields, saw 5\nSkipping line 64: expected 4 fields, saw 5\nSkipping line 74: expected 4 fields, saw 5\nSkipping line 91: expected 4 fields, saw 5\nSkipping line 99: expected 4 fields, saw 5\nSkipping line 140: expected 4 fields, saw 5\nSkipping line 145: expected 4 fields, saw 5\nSkipping line 148: expected 4 fields, saw 5\nSkipping line 152: expected 4 fields, saw 5\nSkipping line 171: expected 4 fields, saw 5\nSkipping line 178: expected 4 fields, saw 5\nSkipping line 197: expected 4 fields, saw 5\nSkipping line 201: expected 4 fields, saw 5\nSkipping line 203: expected 4 fields, saw 5\nSkipping line 213: expected 4 fields, saw 5\nSkipping line 216: expected 4 fields, saw 5\n'


In [21]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[a-zA-Z]')
_df0.head()

,Language,Tweet
0,es,Pedazo de tarta k me e kurrao!!!! Ske k buena ...
1,es,No hace falta que te digan que vas a morir par...
2,es,Empieza mi findeeeee :))))
3,es,Próxima parada: Carnaval del Toro de Ciudad Ro...
4,es,Graniza y tal.


In [22]:
for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [23]:
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if pattern.match(letter): #letter.isalpha() or 
            exec('if \'{0}\' not in {1}Alphabets.keys():\n\
                     {2}Alphabets[letter] = {3}Size\n\
                     {4}Size += 1'.format(letter, language, language, language, language))
            sentence = sentence + letter
        else:
            sentence = sentence + ' '
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

Reading time (hh:mm:ss.ms) 0:00:31.511661


In [24]:
print(enSize)
print(enAlphabets)
print(esSize)
print(esAlphabets)
print(euSize)
print(euAlphabets)
print(caSize)
print(caAlphabets)
print(ptSize)
print(ptAlphabets)
print(glSize)
print(glAlphabets)

52
{'I': 0, 'M': 1, 'C': 2, 'O': 3, 'N': 4, 'F': 5, 'U': 6, 'S': 7, 'E': 8, 'D': 9, 'W': 10, 'H': 11, 'A': 12, 'T': 13, 'R': 14, 'L': 15, 'J': 16, 'j': 17, 'u': 18, 's': 19, 't': 20, 'i': 21, 'n': 22, 'b': 23, 'e': 24, 'r': 25, 'a': 26, 'f': 27, 'o': 28, 'd': 29, 'y': 30, 'p': 31, 'h': 32, 'P': 33, 'm': 34, 'l': 35, 'c': 36, 'Q': 37, 'Z': 38, 'B': 39, 'z': 40, 'v': 41, 'g': 42, 'k': 43, 'w': 44, 'Y': 45, 'x': 46, 'K': 47, 'V': 48, 'X': 49, 'q': 50, 'G': 51}
52
{'P': 0, 'e': 1, 'd': 2, 'a': 3, 'z': 4, 'o': 5, 't': 6, 'r': 7, 'k': 8, 'm': 9, 'u': 10, 'S': 11, 'b': 12, 'n': 13, 's': 14, 'y': 15, 'l': 16, 'h': 17, 'g': 18, 'J': 19, 'j': 20, 'p': 21, 'c': 22, 'O': 23, 'L': 24, 'f': 25, 'D': 26, 'X': 27, 'N': 28, 'q': 29, 'i': 30, 'v': 31, 'E': 32, 'x': 33, 'C': 34, 'T': 35, 'R': 36, 'G': 37, 'M': 38, 'A': 39, 'I': 40, 'V': 41, 'H': 42, 'F': 43, 'K': 44, 'W': 45, 'Q': 46, 'Z': 47, 'B': 48, 'Y': 49, 'U': 50, 'w': 51}
52
{'A': 0, 'n': 1, 'd': 2, 'e': 3, 'r': 4, 'R': 5, 'u': 6, 'i': 7, 'z': 8, 

In [25]:
for language in languages:
    exec("%sModel=np.zeros(shape=((%sSize+1),(%sSize+1))) " %(language, language, language))
#     exec("%sModelFinal=np.zeros(shape=((%sSize+1),(%sSize+1))) " %(language, language, language))
#     exec("%sCount=0" %(language))
#     exec("%sEachWordCount=dict.fromkeys(%sAlphabets, 0)" %(language, language)) #needed for bigrams

In [26]:
print('EN',enModel, enModel.shape)
print('ES',esModel, esModel.shape)
print('EU',euModel, euModel.shape)
print('CA',caModel, caModel.shape)
print('PT',ptModel, ptModel.shape)
print('gl',glModel, glModel.shape)

EN [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (53, 53)
ES [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (53, 53)
EU [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (53, 53)
CA [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (53, 53)
PT [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (53, 53)
gl [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (

In [27]:
# a = dict.fromkeys(a, 0)
# print(trainDict)

#### For Unigrams

In [28]:
# start_time = datetime.now()
# delta = 0.5
# for language, tweets in trainDict.items():
#     for tweet in tweets:
#         for i in range(len(tweet)-1):
#             first = tweet[i]
#             exec('index = %sAlphabets[first]'%(language)) #index = esAlphabets[first] #get index of the character from the language dictionary
#             exec('%sModel[index] += 1'%(language)) #np.add.at(esModel, [index], 1) #increment that index in the language model
# for language in languages:
#     exec('%sModel = %sModel/%sModel.sum()'%(language, language, language)) #divide all the values by the sum of the row
#     exec('%sModel = np.log10(%sModel)'%(language, language))
# print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

#### For Bigrams

In [30]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            if (not first.isspace()) and (not second.isspace()):
                exec('firstIndex = %sAlphabets[first]'%(language))
                exec('secondIndex = %sAlphabets[second]'%(language))
                exec('%sModel[firstIndex][secondIndex] += 1'%(language))
for language in languages:
    exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
    exec('%sModel = %sModel/%sModel.sum(axis=1)'%(language, language, language)) #divide all the values by the sum of the row
    exec('%sModel = np.log10(%sModel)'%(language, language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

Training time (hh:mm:ss.ms) 0:00:40.977785


In [31]:
for language in languages:
    exec('print(\'%s\', %sModel)'%(language, language))
#     exec('print(\'%s\', %sModelFinal)'%(language, language))

# print('EN',enModel, enModel.shape)
# print('EN',enModel, enModel.shape)
# print('ES',esModel, esModel.shape)
# print('ES',esModel, esModel.shape)
# print('EU',euModel, euModel.shape)
# print('EU',euModel, euModel.shape)
# print('CA',caModel, caModel.shape)
# print('PT',ptModel, ptModel.shape)
# print('gl',glModel, glModel.shape)

en [[-2.2460059  -1.52720012 -1.30706795 ... -1.86332286 -1.79339122
  -1.72427587]
 [-1.90358322 -2.15044941 -2.70500796 ... -2.34044411 -1.5245459
  -1.72427587]
 [-1.90358322 -2.84941941 -2.00603795 ... -2.34044411 -2.161368
  -1.72427587]
 ...
 [-2.94497591 -2.84941941 -2.2278867  ... -2.34044411 -2.161368
  -1.72427587]
 [-1.9907334  -2.15044941 -2.70500796 ... -2.34044411 -2.63848926
  -1.72427587]
 [-2.94497591 -2.84941941 -2.70500796 ... -2.34044411 -2.63848926
  -1.72427587]]
es [[-1.83661251 -2.26502873 -3.50061149 ... -1.53258139 -1.98571698
  -1.72427587]
 [-1.80482068 -1.67995813 -1.23689654 ... -1.92255247 -1.40325255
  -1.72427587]
 [-2.143175   -0.74900754 -2.46987688 ... -2.21061749 -1.85443807
  -1.72427587]
 ...
 [-2.21572566 -3.71586412 -3.15818881 ... -1.70070372 -2.17477322
  -1.72427587]
 [-2.69284692 -2.61812767 -3.44625383 ... -2.11884711 -1.69765196
  -1.72427587]
 [-3.7342396  -5.03808342 -4.67670275 ... -3.44106641 -3.2161659
  -1.72427587]]
eu [[-0.95424251

In [ ]:
# glSum = glModel.sum()
# ptSum = ptModel.sum()
# caSum = caModel.sum()
# euSum = euModel.sum()
# esSum = esModel.sum()
# enSum = enModel.sum()

In [ ]:
# print('enSum', enSum ,'\nesSum', esSum ,'\neuSum', euSum ,'\ncaSum', caSum ,'\nptSum', ptSum, '\nglSum', glSum)

### Plotting the model

In [ ]:
# print(enModel)
row = enModel[0:1]
# print('row: ', row)
# y = np.zeros(shape=((row.shape)))
colum = enModel[0][0]
# print('column', colum)

x = row
y = np.full((row.shape), row[0][0])
x = x.flatten()
y = y.flatten()
# print(x.shape)
# print(y.shape)
xAxis = enAlphabets.keys()
# print(xAxis)

# print('x:\n',x)
# print('y:\n ',y)
area = (10 * np.random.rand(x.shape[0]))**2
colors = np.random.rand(x.shape[0])
plt.scatter(x, y, s=area, c=colors, alpha=0.3)

In [ ]:
N = 10
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
area = (30 * np.random.rand(N))**2  # 0 to 15 point radii
print(x.shape)
print(y.shape)
plt.scatter(x, y, s=area, c=colors, alpha=0.5)
plt.show()

In [ ]:
fig = plt.figure()  # an empty figure with no axes
fig.suptitle('First Trial')  # Add a title so we know which it is
plt.scatter(x, y, s=area, c=colors, alpha=0.3)
plt.xlabel('x label')
plt.ylabel('y label')

plt.show()

## Testing the models

In [32]:
df = pd.read_csv('../OriginalDataSet/test-tweets-given.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=100)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
_df0.head()

,TweetID,Language,Tweet
0,439381946939232257,es,Foto antes de LA FOTO. #orla @ Universidad de ...
1,439389330533273600,es,@Nagore_Robles Que mal lo vas a pasar viendo a...
2,439392779786285056,es,"Hay cosas complicadas, y luego está quitarse l..."
3,439393603845373952,es,"Me duele todo, que asco."
4,439395306464346112,eu,Ta aste txuriyakin hastekoo bizitzako parriaak...


In [33]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        second = tweet[i+1]
        for language in languages:
            exec('if ( (not pattern.match(first)) or (not pattern.match(second)) ):\n\
    prob = 0\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    prob = {lang}Model[-1][-1]\n\
elif (second not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1]\n\
elif (first not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang=language))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong')

439381946939232257    ca    -4.86E+01    es wrong
439389330533273600    es    -8.50E+01    es correct
439392779786285056    es    -4.58E+01    es correct
439393603845373952    es    -1.39E+01    es correct
439395306464346112    eu    -4.16E+01    eu correct
439403680245493760    en    -2.05E+01    es wrong
439407703358967808    es    -8.57E+01    es correct
439408603167211520    es    -6.13E+01    es correct
439410377286492160    en    -6.13E+01    es wrong
439411327971643392    es    -8.35E+01    es correct
439414230413217792    es    -7.46E+01    es correct
439415113045778432    eu    -2.22E+01    eu correct
439420783698640896    es    -2.24E+01    es correct
439428207222484992    es    -1.32E+01    es correct
439435803983634432    es    -3.81E+01    es correct
439437804352057344    es    -2.21E+01    es correct
439438406117232640    es    -4.30E+01    es correct
439449097658195968    es    -9.38E+01    es correct
439455534304210944    eu    -6.29E+01    eu correct
439460634212970496

In [37]:

str = 'if vocab==\'2\':\n\
    if (not first.islapha()) and (not second.isalpha()):\n\
        prob = 0\n\
else:\n\
    if ( (not pattern.match(first)) or (not pattern.match(second)) ):\n\
        prob = 0\n\
if ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    prob = {lang}Model[-1][-1]\n\
elif (second not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1]\n\
elif (first not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en')#, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang)
print(str)

if vocab=='2':
    if (not first.islapha()) and (not second.isalpha()):
        prob = 0
else:
    if ( (not pattern.match(first)) or (not pattern.match(second)) ):
        prob = 0
if ( (first not in enAlphabets.keys()) and (second not in enAlphabets.keys()) ):
    prob = enModel[-1][-1]
elif (second not in enAlphabets.keys()):
    index = enAlphabets[first]
    prob = enModel[index][-1]
elif (first not in enAlphabets.keys()):
    index = enAlphabets[second]
    prob = enModel[-1][index]
else:
    firstIndex = enAlphabets[first]
    secondIndex = enAlphabets[second]
    prob = enModel[firstIndex][secondIndex]
enProb = prob + enProb



In [ ]:
a = np.arange(64.0).reshape((4,4,4))
print('before',a)
for x in range(a.shape[0]):
    for y in range(a.shape[1]):
        sumCol = a[x,y,:]
        sumCol = np.add(sumCol, 1)
        sumCol = np.divide(sumCol, sumCol.sum(axis=0))
        sumCol = np.log10(sumCol)
        a[x,y,:] = sumCol
print('after:', a)

In [ ]:
a = np.arange(27.0).reshape((3,3,3))
print('before',a)
X = a[0,0,:]
X = np.add(X, 1)
X = np.divide(X, X.sum(axis=0))
a[0,0,:] = X
print('after',a)

In [ ]:
a = np.arange(9.0).reshape((3,3))
print('before',a)
X = a[0,:]
X = np.add(X, 1)
X = np.divide(X, X.sum(axis=0))
a[0,:] = X
print('after',a)

In [ ]:
a = np.arange(3.0).reshape((3))
print('before',a)
X = a[:]
X = np.add(X, 1)
X = np.divide(X, X.sum(axis=0))
a[:] = X
print('after',a)

In [44]:
str = 'if self.vocab==\'2\':\n\
    if (not first.isalpha()) and (not second.isalpha()):\n\
        prob = 0\n\
else:\n\
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) ):\n\
        prob = 0\n\
if ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    prob = {lang}Model[-1][-1]\n\
elif (second not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1]\n\
elif (first not in {lang}Alphabets.keys()):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en')
print(str)

if self.vocab=='2':
    if (not first.isalpha()) and (not second.isalpha()):
        prob = 0
else:
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) ):
        prob = 0
if ( (first not in enAlphabets.keys()) and (second not in enAlphabets.keys()) ):
    prob = enModel[-1][-1]
elif (second not in enAlphabets.keys()):
    index = enAlphabets[first]
    prob = enModel[index][-1]
elif (first not in enAlphabets.keys()):
    index = enAlphabets[second]
    prob = enModel[-1][index]
else:
    firstIndex = enAlphabets[first]
    secondIndex = enAlphabets[second]
    prob = enModel[firstIndex][secondIndex]
enProb = prob + enProb

